In [1]:
! ls code

Dockerfile  docker-compose.yml	orig-compose.yaml
Makefile    myapp.py		requirements.txt


In [2]:
! cat code/myapp.py


from flask import Flask
from redis import Redis

app = Flask(__name__)
redis = Redis(host='redis', port=6379)

@app.route('/')
def hello():
    count = redis.incr('hits')
    return 'Hello World! I have been seen {} times.\n'.format(count)

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000, debug=True)




In [3]:
! cat code/requirements.txt

flask
redis


In [4]:
! cat code/Dockerfile

FROM python:3.4-alpine
ADD . /code
WORKDIR /code
RUN pip install -r requirements.txt
CMD ["python", "myapp.py"]




In [5]:
! cat code/Makefile

Image='tbutzer/stackdemo'

build:
	docker build --tag ${Image} .

publish: ## Send this image to docker hub!
	sudo docker login -u tbutzer
	sudo docker push ${Image}

gnufix:
	sudo apt install gnupg2 pass

test:
	for i in 1 2 3 4 5 6\ ; do\
		curl localhost:8000 ; \
	done

shell:
	docker run --rm -it \
		-v /data3/butzer:/data \
		--name rose ${Image} /bin/bash


In [6]:
! cat code/docker-compose.yml

version: '3'

services:
  web:
    image: tbutzer/stackdemo
    build: .
    ports:
      - "8000:8000"
  redis:
    image: redis:alpine
  tony:
    image: tbutzer/bash-tester


In [20]:
!  docker stack deploy --compose-file code/docker-compose.yml stackdemo

Ignoring unsupported options: build

Updating service stackdemo_web (id: tqrtravuiucpthxbd3k2qjuxs)
Updating service stackdemo_redis (id: 6j18qgcudthmerrteql6p3y5f)
Updating service stackdemo_tony (id: rzrmfxzx2phxpcdrsv4y8nm6k)


In [21]:
!  docker service ls

ID                  NAME                MODE                REPLICAS            IMAGE                        PORTS
6j18qgcudthm        stackdemo_redis     replicated          1/1                 redis:alpine                 
rzrmfxzx2phx        stackdemo_tony      replicated          0/1                 tbutzer/bash-tester:latest   
tqrtravuiucp        stackdemo_web       replicated          1/1                 tbutzer/stackdemo:latest     *:8000->8000/tcp


In [22]:
!  docker service ps stackdemo_tony

ID                  NAME                   IMAGE                        NODE                DESIRED STATE       CURRENT STATE              ERROR               PORTS
u8lo7cidm69y        stackdemo_tony.1       tbutzer/bash-tester:latest   masterswarm         Running             Preparing 11 seconds ago                       
xc32tai4ccrh         \_ stackdemo_tony.1   tbutzer/bash-tester:latest   ship1               Shutdown            Shutdown 7 seconds ago                         


In [23]:
! docker exec -it `docker ps |grep bash-tester | awk '{print $1}'` curl web:8000

Hello World! I have been seen 1 times.


In [24]:
! docker exec -it `docker ps |grep bash-tester | awk '{print $1}'` curl stackdemo_web:8000

Hello World! I have been seen 2 times.


In [25]:
! docker exec -it `docker ps |grep bash-tester | awk '{print $1}'`  host stackdemo_web

stackdemo_web has address 10.0.1.2


In [26]:
! docker exec -it `docker ps |grep bash-tester | awk '{print $1}'`  host redis

redis has address 10.0.1.5


In [27]:
! docker network ls

NETWORK ID          NAME                DRIVER              SCOPE
5fe4813d0710        bridge              bridge              local
7fe36f16453b        docker_gwbridge     bridge              local
c621c6196cce        host                host                local
rmp9lxy32v1b        ingress             overlay             swarm
0d22e1d79f59        none                null                local
iuxon8c7gauj        stackdemo_default   overlay             swarm


In [28]:
! docker exec -it `docker ps |grep bash-tester | awk '{print $1}'`  host 10.0.2.7

7.2.0.10.in-addr.arpa domain name pointer ip-10-0-2-7.us-west-2.compute.internal.


# Try this in a terminal

1. redis-cli -h redis
2. ping


In [43]:
! docker exec -it `docker ps |grep bash-tester | awk '{print $1}'` curl web:8000

Hello World! I have been seen 36 times.


In [42]:
! docker exec -it `docker ps |grep bash-tester | awk '{print $1}'` redis-cli -h redis ping

PONG


In [29]:
! echo docker exec -it `docker ps |grep bash-tester | awk '{print $1}'` bash

docker exec -it dfa920542386 bash


In [30]:
! docker service inspect stackdemo_redis

[
    {
        "ID": "6j18qgcudthmerrteql6p3y5f",
        "Version": {
            "Index": 50
        },
        "CreatedAt": "2020-04-09T01:15:56.314122647Z",
        "UpdatedAt": "2020-04-09T01:20:53.866329115Z",
        "Spec": {
            "Name": "stackdemo_redis",
            "Labels": {
                "com.docker.stack.image": "redis:alpine",
                "com.docker.stack.namespace": "stackdemo"
            },
            "TaskTemplate": {
                "ContainerSpec": {
                    "Image": "redis:alpine@sha256:511275ddf4c4582e296f6fca803df853a8ab2b733a704c44c498f0a6a90e5ba0",
                    "Labels": {
                        "com.docker.stack.namespace": "stackdemo"
                    },
                    "Privileges": {
                        "CredentialSpec": null,
                        "SELinuxContext": null
                    },
                    "StopGracePeriod": 10000000000,
                    "DNSConfig": {},
                    "Isol

In [32]:
! docker service inspect stackdemo_redis | egrep -i 'ip|addr'

                "Mode": "vip"
                "Mode": "vip"
                "Mode": "vip"
            "VirtualIPs": [
                    "Addr": "10.0.1.5/24"


In [33]:
! docker exec -it `docker ps |grep bash-tester | awk '{print $1}'`  host redis

redis has address 10.0.1.5


In [34]:
! sudo netstat -anlp | grep 8000

tcp6       0      0 :::8000                 :::*                    LISTEN      5542/dockerd        


https://dev.to/realabbas/explain-redis-like-i-m-five-4anm


Redis is an open source (BSD licensed), 
- in-memory data structure store, 
- used as a database, cache and message broker. -
- It supports data structures such as 
    - strings, 
    - hashes, -
    - lists, 
    - sets, 
    sorted sets with range queries, bitmaps, hyperloglogs, geospatial indexes with radius queries and streams. Redis has built-in replication, Lua scripting, LRU eviction, transactions and different levels of on-disk persistence, and provides high availability via Redis Sentinel and automatic partitioning with Redis Cluster. 

In [35]:
! docker service  ps stackdemo_redis

ID                  NAME                IMAGE               NODE                DESIRED STATE       CURRENT STATE           ERROR               PORTS
4suuxg1we8zg        stackdemo_redis.1   redis:alpine        masterswarm         Running             Running 5 minutes ago                       


In [36]:
! docker service ls | awk '{print $5, $6}'

IMAGE PORTS
redis:alpine 
tbutzer/bash-tester:latest 
tbutzer/stackdemo:latest *:8000->8000/tcp


In [37]:
! curl localhost:8000

Hello World! I have been seen 3 times.


In [38]:
%%bash

for i in {1..20} ; do
{
    curl localhost:8000 
} ;
done

Hello World! I have been seen 4 times.
Hello World! I have been seen 5 times.
Hello World! I have been seen 6 times.
Hello World! I have been seen 7 times.
Hello World! I have been seen 8 times.
Hello World! I have been seen 9 times.
Hello World! I have been seen 10 times.
Hello World! I have been seen 11 times.
Hello World! I have been seen 12 times.
Hello World! I have been seen 13 times.
Hello World! I have been seen 14 times.
Hello World! I have been seen 15 times.
Hello World! I have been seen 16 times.
Hello World! I have been seen 17 times.
Hello World! I have been seen 18 times.
Hello World! I have been seen 19 times.
Hello World! I have been seen 20 times.
Hello World! I have been seen 21 times.
Hello World! I have been seen 22 times.
Hello World! I have been seen 23 times.


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    39  100    39    0     0  13000      0 --:--:-- --:--:-- --:--:-- 13000
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    39  100    39    0     0  13000      0 --:--:-- --:--:-- --:--:-- 19500
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    39  100    39    0     0  19500      0 --:--:-- --:--:-- --:--:-- 19500
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    39  100    39    0     0   7800      0 --:--:-- --:--:-- --:--:--  9750
  % Total    % Received % Xferd  Average Speed   Tim

In [39]:
%%bash

CONTAINER=`docker ps |grep bash-tester | awk '{print $1}'`

echo ${CONTAINER}

for i in {1..12} ; do
{
    docker exec ${CONTAINER} curl web:8000
} ;
done


dfa920542386
Hello World! I have been seen 24 times.
Hello World! I have been seen 25 times.
Hello World! I have been seen 26 times.
Hello World! I have been seen 27 times.
Hello World! I have been seen 28 times.
Hello World! I have been seen 29 times.
Hello World! I have been seen 30 times.
Hello World! I have been seen 31 times.
Hello World! I have been seen 32 times.
Hello World! I have been seen 33 times.
Hello World! I have been seen 34 times.
Hello World! I have been seen 35 times.


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    40  100    40    0     0   5402      0 --:--:-- --:--:-- --:--:--  5714
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    40  100    40    0     0   5465      0 --:--:-- --:--:-- --:--:--  5714
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    40  100    40    0     0   5423      0 --:--:-- --:--:-- --:--:--  5714
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    40  100    40    0     0   5377      0 --:--:-- --:--:-- --:--:--  5714
  % Total    % Received % Xferd  Average Speed   Tim